<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Questions-for-participants" data-toc-modified-id="Questions-for-participants-0.0.0.1"><span class="toc-item-num">0.0.0.1&nbsp;&nbsp;</span><strong>Questions for participants</strong></a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Importing-Data-and-Libraries" data-toc-modified-id="Importing-Data-and-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing Data and Libraries</a></span></li><li><span><a href="#Data-summary" data-toc-modified-id="Data-summary-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data summary</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Q4.-The-average-number-of-finished-challenges-for-each-division-shows-the-division-with-the-highest-and-lowest-average-challenges." data-toc-modified-id="Q4.-The-average-number-of-finished-challenges-for-each-division-shows-the-division-with-the-highest-and-lowest-average-challenges.-2.0.0.1"><span class="toc-item-num">2.0.0.1&nbsp;&nbsp;</span>Q4. The average number of finished challenges for each division shows the division with the highest and lowest average challenges.</a></span></li></ul></li><li><span><a href="#[graph-4-1]-Violin-plot-about-Challenges-by-ladder" data-toc-modified-id="[graph-4-1]-Violin-plot-about-Challenges-by-ladder-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>[graph 4-1] Violin plot about Challenges by ladder</a></span><ul class="toc-item"><li><span><a href="#Q5.-Show-dependency-between-level-and-class-of-dead-characters.-Only-for-HC-divisions." data-toc-modified-id="Q5.-Show-dependency-between-level-and-class-of-dead-characters.-Only-for-HC-divisions.-2.0.1.1"><span class="toc-item-num">2.0.1.1&nbsp;&nbsp;</span>Q5. Show dependency between level and class of dead characters. Only for HC divisions.</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#5-1.-describes-and-data-preprocessing" data-toc-modified-id="5-1.-describes-and-data-preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>5-1. describes and data preprocessing</a></span><ul class="toc-item"><li><span><a href="#(1)-ratio-of-dead-characters" data-toc-modified-id="(1)-ratio-of-dead-characters-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>(1) ratio of dead characters</a></span></li><li><span><a href="#(2)-level-distribution" data-toc-modified-id="(2)-level-distribution-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>(2) level distribution</a></span></li><li><span><a href="#(3)-preprocessing-the-data" data-toc-modified-id="(3)-preprocessing-the-data-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>(3) preprocessing the data</a></span><ul class="toc-item"><li><span><a href="#get-dummies" data-toc-modified-id="get-dummies-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>get dummies</a></span></li></ul></li></ul></li><li><span><a href="#5-1.-correlation" data-toc-modified-id="5-1.-correlation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>5-1. correlation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#result1:" data-toc-modified-id="result1:-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>result1:</a></span></li></ul></li></ul></li><li><span><a href="#5-2.-Average-level-of-each-class" data-toc-modified-id="5-2.-Average-level-of-each-class-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>5-2. Average level of each class</a></span><ul class="toc-item"><li><span><a href="#5-2-1.-Average-level" data-toc-modified-id="5-2-1.-Average-level-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>5-2-1. Average level</a></span></li></ul></li></ul></div>

# POE user analyzing project
###### **Data reference and questions are here: https://www.kaggle.com/datasets/gagazet/path-of-exile-league-statistic**

---
Hi, I'm Angela. 😊 I'm a novice in data analysis.  I'm a novice in data analysis. I'm interested in the video games, mobility, culture, and content industry. I hope to become a data scientist with anomaly detection and anti-cheating. I hope that my analysis will have a positive impact on the world at least a little bit.  

This is my first Kaggle-data analyzing project on my own. I played(?) with this data for almost a month or more freely, and I finish it to start to answer these questions partially. Finally, I can close the analysis joyfully.
I analyzed this data set with my thoughts and what I learned (as if I were a game analyst 😆).  
_ps. English may seem a little awkward because it is not my first language. I'm sorry. And I used the word 'average' more than the word 'mean' because I wanted to communicate more clearly in my hectic writing. 'mean' is so mean to 'average', 'mean' means 'mean' and... just joking._ 🤪

---
##### **Questions for participants**

1. A total number of players in each division, usage of each class in descending order.
2. Some of the players streaming their game (twitch column). Do they play better than people, who does not?
3. Predict chance to be at top 30 in each division, if we are Necromancer. With and without stream.
4. Average number of finished challanges for each division, show division with highest and lowest average challanges.
5. Show dependency between level and class of died characters. Only for HC divisions.


## Importing Data and Libraries
---

In [ ]:
import pandas as pd
import numpy as np

import time as time
import matplotlib.pyplot as plt
import seaborn as sns

# import sklearn as sk
# import tensorflow as tf
# import keras

np.random.seed(0)
# tf.random.set_seed(0)

# version check
print(sns.__version__)
print(pd.__version__)
print(np.__version__)
# print(sk.__version__)
# print(tf.__version__)
# print(keras.__version__)

data = pd.read_csv("poe_stats.csv")

In [ ]:
df = data
df.head()

## Data summary

In [ ]:
# options about float

pd.options.display.float_format = '{:,.2f}'.format
df.describe()

In [ ]:
df.describe(include = 'object')

In [ ]:
df.info()

# 4. The average number of "finished" challenges 
---
##### Q4. The average number of finished challenges for each division shows the division with the highest and lowest average challenges.

This question is really clear, thankfully.  
I'll separate the division and calculate the average challenges.  


In [ ]:
# simple max and min
df.groupby(by = 'ladder')[['challenges']].agg(['max', 'min'])

In [ ]:
# simple max and min
df.groupby(by = 'ladder')[['challenges']].agg(['max', 'min'])

#### [graph 4-1] Violin plot about Challenges by ladder

In [ ]:
# visualizing the average
plt.rcParams["figure.figsize"] = (6, 6)
plt.xticks(rotation = 45)
sns.violinplot(data = df.sort_values(by = 'ladder'), x = 'ladder', y = 'challenges', palette = 'Paired')
plt.show()

# 5. Is the class of the dead characters related to level?
---
##### Q5. Show dependency between level and class of dead characters. Only for HC divisions.

The dead column has a meaning in only hardcore modes.  
That means maybe this: the goal of the hardcore mode is survival with the best playing, and maybe alive characters can restart or continue the game.  

I understood this question as the relation between the dead characters' class and the level.   
Because we can think that the death is related just level(user's ability) or maybe the class is the problem.  
If there is a dependency between level and class in dead characters, we can treat that some particular class is **preferred** because of the difficulty or something like that, regardless of the class's performance(because it's already ended games).

The methods which can get the dependency,  
    1. get correlation between level and class.  
    2. compare the average level by each class (+ t-test)

And I imagine also this : make a machine learning model with class and level, and watch whether the model can predict the class just with level! I'm little bit sorry for the computer but the computer can't do that.


## 5-1. describes and data preprocessing
### (1) ratio of dead characters

In [ ]:
# dataframe contains only hardcore modes
df_hc = df.loc[df['ladder'].isin(['SSF Harbinger HC', 'Hardcore Harbinger'])]

# dead characters rate
df_hc.pivot_table(index = 'ladder', values = 'dead')

### (2) level distribution

In [ ]:
df_hc['level'].describe()

In [ ]:
plt.figure(figsize = (6, 3))
sns.histplot(x = df_hc['level'], palette = 'Paired', kde = True)
plt.title("level of dead characters in Hardcore mode")
plt.show()

Many people already know that **the minimum level of users in Hardcore mode is 53** before reaching this question.  
This is quite important because the scale is regarded as big. After all, because it starts at 53.  
So I applied **Min-Max scaler** in this case because the minimum and maximum are settled.  

### (3) preprocessing the data

In [ ]:
# dead
df_dead = df_hc.loc[df['dead'] == True][['class', 'level']]
df_dead

In [ ]:
# Min-Max Scaling
Xmin = min(df_dead['level'])
Xmax = max(df_dead['level'])
scaled = (df_dead['level'] - Xmin) / (Xmax - Xmin)
df_dead['scaled_level'] = scaled
df_dead.head(5)

#### get dummies

In [ ]:
df_deads_onehot = pd.get_dummies(df_dead, drop_first = False)
df_deads_onehot.head(5)

## 5-1. correlation

In [ ]:
df_deads_onehot.corr().round(3)

In [ ]:
mask = np.zeros_like(df_deads_onehot.corr())
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize = (10, 10))
sns.heatmap(data = df_deads_onehot.corr(), cmap = 'RdBu', mask=mask, square = True, vmin = -1, vmax = 1)
plt.title('Correlation matrix')
plt.show()

#### result1:
That seems there's no correlation between the level and the class, but the data indeed became scarce data after getting dummies.  
In this case, we can use polyserial correlation analysis.

`stats.pointbiserialr` need bools array... 'biserial'... this is nominal...

## 5-2. Average level of each class 

### 5-2-1. Average level

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
plt.xticks(rotation = 90)
plt.ylim(50, 105)
df_hc_lev_mean = df_dead.groupby('class')[['level']].mean()
sns.barplot(data = df_hc_lev_mean, x = df_hc_lev_mean.index, y = 'level') # 신뢰구간 표시 못하나 그냥 boxplot그려야 하나
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
plt.xticks(rotation = 90)
df_hc_lev_mean = df_dead.groupby('class')[['scaled_level']].mean()
sns.barplot(data = df_hc_lev_mean, x = df_hc_lev_mean.index, y = 'scaled_level')
plt.show()

아니 스케일링을 하는 이유가 없어지는데... 그냥 해야겠다. 다 지우고,,,, 그냥 기록만 남기고...

그래프를 보고 알 수 있는 것: 듀엘리스트, 머러더, 레인저, 사이온, 셰도우, 템플러, 위치는 평균 레벨이 현저히 낮다.  
아마 이 모든 클래스를 데리고 ANOVA를 하면 분명 다르다고 할 것이고, 전부 사후검정을 하게되면 이 클래스들에서 차이가 나타날 것이다.  
일부 클래스에서 낮은 것은 사실이나 이것이 꼭 레벨과 관련이 있다고 할 수 있는 것은 아니다.  
왜냐하면 이미 언급했듯, 이 모드의 최소 레벨은 53으로, 경험이 풍부한 사람이 부 캐릭터를 53까지 키워서 하드코어 모드에서 기록을 세웠을 가능성이 다분하며, 그 직업들이 하드코어모드에서 충분히 인기있는 클래스들이었다면 더욱더 이 시나리오가 맞을 가능성이 높다.   

그리고 해야할 것 
- 클래스별 평균 구하기
- 평균이 다른지 ANOVA 하기